# Foil

The :class:`~pylabrobot.liquid_handling.backends.hamilton.STAR.STAR` backend includes special utilities for working with foil-sealed plates, specifically:

1. a function to pierce foil before aspirating from the plate, and
2. a function to keep the plate down while moving the channels up to avoid lifting the plate.

## Example setup

```{note}
While this example uses high volume tips, it _might_ be possible to use other tip types to pierce the foil. However, 50uL tips are very soft and probably can't be used.
```

In [2]:
from pylabrobot.liquid_handling import LiquidHandler, STAR
from pylabrobot.resources import STARLetDeck
from pylabrobot.resources import (
  TIP_CAR_480_A00,
  PLT_CAR_L5AC_A00,
  HT,
  AGenBio_4_wellplate_Vb
)

star = STAR()
lh = LiquidHandler(backend=star, deck=STARLetDeck())
await lh.setup()

# assign a tip rack
tip_carrier = TIP_CAR_480_A00(name="tip_carrier")
tip_carrier[1] = tip_rack = HT(name="tip_rack")
lh.deck.assign_child_resource(tip_carrier, rails=1)

# assign a plate
plt_carrier = PLT_CAR_L5AC_A00(name="plt_carrier")
plt_carrier[0] = plate = AGenBio_4_wellplate_Vb(name="plate")
lh.deck.assign_child_resource(plt_carrier, rails=10)

## Breaking the foil before using a plate

It is important to break the foil before aspirating because tiny foil pieces can stuck in the tip, drastically changing the liquid handling characteristics.

In this example, we will use an 8 channel workcell and use the inner 6 channels for breaking the foil and then aspirating. We will use the outer 2 channels to keep the plate down while the inner channels are moving up.

In [3]:
well = plate.get_well("A1")
await lh.pick_up_tips(tip_rack["A1:H1"])

In [4]:
aspiration_channels = [1, 2, 3, 4, 5, 6]
hold_down_channels = [0, 7]
await star.pierce_foil(
  wells=[well],
  piercing_channels=aspiration_channels,
  hold_down_channels=hold_down_channels,
  move_inwards=4,
  one_by_one=False,
)

In [5]:
await lh.return_tips()

![gif of piercing foil](./img/pierce_foil.gif)

## Holding the plate down

Holding the plate down while moving channels up after aspiration consists of two parts:
1. Making the channels stay down after a liquid handling operation has finished. By default, STAR will move channels up to traversal height.
2. Putting two channels on the edges of the plate to hold it down, while moving the other channels up.

In [11]:
await lh.pick_up_tips(tip_rack["A2:H2"])

In [12]:
num_channels = len(aspiration_channels)
await lh.aspirate(
  [well]*num_channels, vols=[100]*num_channels, use_channels=aspiration_channels,

  # aspiration parameters (backend_kwargs)
  min_z_endpos=well.get_absolute_location(z="cavity_bottom").z, # z end position: where channels go after aspiration
  surface_following_distance=0, # no moving in z dimension during aspiration
  pull_out_distance_transport_air=[0] * num_channels # no moving up to aspirate transport air after aspiration
)

await star.step_off_foil(
  well,
  front_channel=7,
  back_channel=0,
  move_inwards=5,
)

In [13]:
await lh.return_tips()

![gif of holding down foil](./img/step_off_foil.gif)